
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering \[Provider]

## Overview
This demo showcases advanced data governance patterns for Delta Sharing using:

- **Dynamic Views**: Row-level security based on recipient identity
- **Partition Filtering**: Column-level data masking and regional filtering
- **Data Classification**: Automatic PII detection and protection
- **Conditional Logic**: Context-aware data access controls

These patterns enable providers to share the same underlying dataset with multiple recipients while ensuring each recipient only sees data appropriate for their access level, region, or business relationship.

**Provider Notebook (This Notebook):** Create source data with sensitive information, implement dynamic views with recipient-specific filtering, configure partition-based access controls, and demonstrate data masking techniques.

**Recipient Notebook:** Access shared views as different recipient types and observe how data appears differently based on access controls.

### Learning Objectives
By the end of this demo, you will be able to:
1. Implement row-level security using dynamic views and recipient context
2. Create partition-based filtering for regional or organizational boundaries
3. Apply column-level security with data masking and redaction
4. Use conditional logic to vary data access based on recipient identity
5. Combine multiple security patterns for comprehensive data governance
6. Monitor and audit data access patterns across different recipients

## Background

**Scenario:**
You are "SecureBank Corp" with a comprehensive customer database containing:
- Customer demographics and contact information
- Account balances and transaction history
- Credit scores and risk assessments
- Regional regulatory data

You need to share different views of this data with:
- **Regional Partners**: Only customers from their geographic region
- **Marketing Agencies**: Demographics without financial data
- **Risk Analytics Firms**: Anonymized financial patterns
- **Compliance Auditors**: Full access with audit trails

Each recipient should see a tailored view based on their business needs and compliance requirements.

In [0]:
%run ./Includes/Demo-Setup-3.2

## Step 1: Create Source Tables with Sensitive Data

Create comprehensive customer data including PII, financial information, and regional data.

In [0]:
-- Create comprehensive customer table with sensitive data
CREATE TABLE IF NOT EXISTS secure_bank.customers (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  ssn STRING,
  date_of_birth DATE,
  address STRING,
  city STRING,
  state STRING,
  country STRING,
  region STRING,
  account_balance DECIMAL(15,2),
  credit_score INT,
  risk_category STRING,
  account_type STRING,
  customer_since DATE,
  is_vip BOOLEAN,
  data_classification STRING
)
USING DELTA
PARTITIONED BY (region, data_classification)
COMMENT 'Customer master data with PII and financial information';

In [0]:
-- Insert sample customer data with various sensitivity levels
INSERT INTO secure_bank.customers VALUES
  -- North American customers
  ('CUST001', 'John', 'Smith', 'john.smith@email.com', '+1-555-0101', '123-45-6789', '1985-03-15', '123 Main St', 'New York', 'NY', 'USA', 'North America', 150000.00, 750, 'Low', 'Premium', '2020-01-15', true, 'Confidential'),
  ('CUST002', 'Sarah', 'Johnson', 'sarah.j@email.com', '+1-555-0102', '987-65-4321', '1990-07-22', '456 Oak Ave', 'Toronto', 'ON', 'Canada', 'North America', 75000.00, 680, 'Medium', 'Standard', '2021-03-10', false, 'Internal'),
  ('CUST003', 'Michael', 'Brown', 'mbrown@email.com', '+1-555-0103', '555-44-3333', '1978-11-08', '789 Pine St', 'Los Angeles', 'CA', 'USA', 'North America', 250000.00, 820, 'Low', 'Private', '2019-05-20', true, 'Restricted'),
  
  -- European customers
  ('CUST004', 'Emma', 'Mueller', 'emma.m@email.com', '+49-123-456789', 'DE-123456789', '1988-02-14', 'Hauptstr 15', 'Berlin', 'BE', 'Germany', 'Europe', 95000.00, 710, 'Medium', 'Standard', '2020-08-12', false, 'Internal'),
  ('CUST005', 'Pierre', 'Dubois', 'pierre.d@email.com', '+33-123-456789', 'FR-987654321', '1982-09-30', '12 Rue de Rivoli', 'Paris', 'IF', 'France', 'Europe', 180000.00, 765, 'Low', 'Premium', '2019-11-03', true, 'Confidential'),
  ('CUST006', 'Sofia', 'Rossi', 'sofia.r@email.com', '+39-123-456789', 'IT-456789123', '1992-06-18', 'Via Roma 25', 'Milan', 'LM', 'Italy', 'Europe', 62000.00, 645, 'High', 'Basic', '2022-01-28', false, 'Internal'),
  
  -- Asia-Pacific customers
  ('CUST007', 'Hiroshi', 'Tanaka', 'hiroshi.t@email.com', '+81-3-1234-5678', 'JP-123456789', '1986-04-12', '1-1-1 Shibuya', 'Tokyo', 'TO', 'Japan', 'Asia Pacific', 320000.00, 800, 'Low', 'Private', '2018-12-15', true, 'Restricted'),
  ('CUST008', 'Li', 'Wei', 'li.wei@email.com', '+86-21-12345678', 'CN-987654321', '1991-10-25', '88 Nanjing Rd', 'Shanghai', 'SH', 'China', 'Asia Pacific', 145000.00, 720, 'Medium', 'Premium', '2020-06-08', false, 'Confidential'),
  ('CUST009', 'Priya', 'Sharma', 'priya.s@email.com', '+91-11-12345678', 'IN-123456789', '1989-12-03', 'MG Road 42', 'Mumbai', 'MH', 'India', 'Asia Pacific', 85000.00, 690, 'Medium', 'Standard', '2021-09-14', false, 'Internal'),
  
  -- Test data for various scenarios
  ('CUST010', 'Anonymous', 'User', 'test@email.com', '+1-000-0000', '000-00-0000', '1980-01-01', 'Test Address', 'Test City', 'TS', 'Test Country', 'North America', 50000.00, 600, 'High', 'Basic', '2023-01-01', false, 'Public');

SELECT COUNT(*) as total_customers FROM secure_bank.customers;

In [0]:
-- Create transaction history table
CREATE TABLE IF NOT EXISTS secure_bank.transactions (
  transaction_id STRING,
  customer_id STRING,
  transaction_date TIMESTAMP,
  amount DECIMAL(15,2),
  transaction_type STRING,
  merchant_category STRING,
  region STRING,
  is_international BOOLEAN,
  risk_score DECIMAL(3,2)
)
USING DELTA
PARTITIONED BY (region)
COMMENT 'Customer transaction history';

In [0]:
-- Insert sample transaction data
INSERT INTO secure_bank.transactions VALUES
  ('TXN001', 'CUST001', '2025-01-15 10:30:00', 1500.00, 'Purchase', 'Retail', 'North America', false, 0.15),
  ('TXN002', 'CUST001', '2025-01-16 14:20:00', -500.00, 'ATM Withdrawal', 'Banking', 'North America', false, 0.05),
  ('TXN003', 'CUST002', '2025-01-15 09:45:00', 2300.00, 'Salary Deposit', 'Employment', 'North America', false, 0.01),
  ('TXN004', 'CUST004', '2025-01-16 16:15:00', 850.00, 'Purchase', 'Travel', 'Europe', true, 0.25),
  ('TXN005', 'CUST005', '2025-01-17 11:30:00', -200.00, 'Bill Payment', 'Utilities', 'Europe', false, 0.02),
  ('TXN006', 'CUST007', '2025-01-15 18:45:00', 5000.00, 'Investment', 'Finance', 'Asia Pacific', false, 0.10),
  ('TXN007', 'CUST008', '2025-01-16 13:20:00', 750.00, 'Purchase', 'Electronics', 'Asia Pacific', false, 0.12),
  ('TXN008', 'CUST003', '2025-01-17 10:10:00', 12000.00, 'Wire Transfer', 'Banking', 'North America', true, 0.30);

SELECT * FROM secure_bank.transactions ORDER BY transaction_date DESC;

## Step 2: Understand Recipient Context in Delta Sharing

Delta Sharing provides recipient context through the `current_recipient()` function, which we'll use for dynamic filtering.

In [0]:
-- First, let's understand how recipient context works
-- When queried through Delta Sharing, current_recipient() returns the recipient name
-- When queried directly, it returns NULL

SELECT 
  current_recipient() as recipient_name,
  current_user() as current_user,
  current_timestamp() as query_time;

-- This will show NULL for recipient_name since we're querying directly

## Step 3: Create Dynamic Views with Recipient-Based Filtering

Create views that automatically filter data based on who is accessing it.

In [0]:
-- View 1: Regional Partners - Only see customers from their region
CREATE OR REPLACE VIEW secure_bank.regional_customers_view AS
SELECT 
  customer_id,
  first_name,
  last_name,
  email,
  city,
  state,
  country,
  region,
  account_type,
  customer_since,
  is_vip,
  -- Conditional data access based on recipient
  CASE 
    WHEN current_recipient() IN ('na_partner', 'north_america_analytics') THEN account_balance
    ELSE NULL 
  END as account_balance,
  CASE 
    WHEN current_recipient() IN ('na_partner', 'north_america_analytics') THEN credit_score
    ELSE NULL 
  END as credit_score
FROM secure_bank.customers
WHERE 
  -- Regional filtering based on recipient
  CASE 
    WHEN current_recipient() IN ('na_partner', 'north_america_analytics') THEN region = 'North America'
    WHEN current_recipient() IN ('eu_partner', 'europe_analytics') THEN region = 'Europe'
    WHEN current_recipient() IN ('apac_partner', 'asia_analytics') THEN region = 'Asia Pacific'
    WHEN current_recipient() = 'global_auditor' THEN true  -- Full access
    ELSE false  -- No access
  END;

In [0]:
-- View 2: Marketing-Safe Customer Data - No PII or financial data
CREATE OR REPLACE VIEW secure_bank.marketing_customers_view AS
SELECT 
  customer_id,
  -- Hash or mask PII for marketing use
  SHA2(CONCAT(first_name, last_name), 256) as customer_hash,
  SUBSTRING(email, 1, 3) || '***@' || SUBSTRING_INDEX(email, '@', -1) as masked_email,
  city,
  state,
  country,
  region,
  -- Age ranges instead of exact dates
  CASE 
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 25 THEN '18-24'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 35 THEN '25-34'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 45 THEN '35-44'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 55 THEN '45-54'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 65 THEN '55-64'
    ELSE '65+'
  END as age_group,
  -- Account tier instead of exact balance
  CASE 
    WHEN account_balance < 50000 THEN 'Basic'
    WHEN account_balance < 100000 THEN 'Standard'
    WHEN account_balance < 200000 THEN 'Premium'
    ELSE 'Private'
  END as account_tier,
  account_type,
  YEAR(customer_since) as customer_since_year,
  is_vip
FROM secure_bank.customers
WHERE 
  -- Marketing recipients only see non-restricted data
  CASE 
    WHEN current_recipient() IN ('marketing_agency_a', 'marketing_agency_b') 
         THEN data_classification IN ('Public', 'Internal')
    WHEN current_recipient() = 'global_auditor' THEN true
    ELSE false
  END;

In [0]:
-- View 3: Risk Analytics - Anonymized financial patterns
CREATE OR REPLACE VIEW secure_bank.risk_analytics_view AS
SELECT 
  -- Anonymized customer identifier
  SHA2(customer_id, 256) as anonymous_customer_id,
  region,
  -- Bucketed financial metrics
  CASE 
    WHEN account_balance < 25000 THEN 'Low'
    WHEN account_balance < 100000 THEN 'Medium'
    WHEN account_balance < 250000 THEN 'High'
    ELSE 'Very High'
  END as balance_category,
  -- Credit score ranges
  CASE 
    WHEN credit_score < 600 THEN 'Poor'
    WHEN credit_score < 650 THEN 'Fair'
    WHEN credit_score < 700 THEN 'Good'
    WHEN credit_score < 750 THEN 'Very Good'
    ELSE 'Excellent'
  END as credit_category,
  risk_category,
  account_type,
  -- Tenure in years
  FLOOR(DATEDIFF(current_date(), customer_since) / 365) as tenure_years,
  is_vip
FROM secure_bank.customers
WHERE 
  -- Risk analytics recipients need broader access but anonymized
  CASE 
    WHEN current_recipient() IN ('risk_analytics_firm', 'credit_bureau') THEN true
    WHEN current_recipient() = 'global_auditor' THEN true
    ELSE false
  END;

In [0]:
-- View 4: Compliance/Audit View - Full access with audit metadata
CREATE OR REPLACE VIEW secure_bank.compliance_audit_view AS
SELECT 
  *,
  current_recipient() as accessed_by,
  current_user() as accessed_by_user,
  current_timestamp() as access_timestamp,
  'FULL_ACCESS_AUDIT' as access_level
FROM secure_bank.customers
WHERE 
  -- Only compliance and audit recipients get full access
  current_recipient() IN ('global_auditor', 'compliance_team', 'regulatory_authority');

## Step 4: Create Transaction Views with Dynamic Filtering

Apply similar patterns to transaction data with additional risk-based filtering.

In [0]:
-- Regional transaction view
CREATE OR REPLACE VIEW secure_bank.regional_transactions_view AS
SELECT 
  t.transaction_id,
  t.customer_id,
  t.transaction_date,
  -- Amount masking based on recipient type
  CASE 
    WHEN current_recipient() IN ('na_partner', 'eu_partner', 'apac_partner') THEN 
      CASE 
        WHEN t.amount > 10000 THEN ROUND(t.amount, -3)  -- Round large amounts
        ELSE t.amount
      END
    WHEN current_recipient() = 'global_auditor' THEN t.amount
    ELSE NULL
  END as amount,
  t.transaction_type,
  t.merchant_category,
  t.region,
  t.is_international,
  -- Risk score only for authorized recipients
  CASE 
    WHEN current_recipient() IN ('risk_analytics_firm', 'global_auditor') THEN t.risk_score
    ELSE NULL
  END as risk_score
FROM secure_bank.transactions t
JOIN secure_bank.customers c ON t.customer_id = c.customer_id
WHERE 
  -- Regional and recipient-based filtering
  CASE 
    WHEN current_recipient() IN ('na_partner', 'north_america_analytics') THEN t.region = 'North America'
    WHEN current_recipient() IN ('eu_partner', 'europe_analytics') THEN t.region = 'Europe'
    WHEN current_recipient() IN ('apac_partner', 'asia_analytics') THEN t.region = 'Asia Pacific'
    WHEN current_recipient() IN ('risk_analytics_firm', 'global_auditor') THEN true
    ELSE false
  END
  -- Additional filtering: exclude high-risk transactions for basic partners
  AND CASE 
    WHEN current_recipient() IN ('na_partner', 'eu_partner', 'apac_partner') 
         THEN t.risk_score <= 0.20
    ELSE true
  END;

## Step 5: Implement Advanced Security Patterns

Create more sophisticated access controls using functions and additional metadata.

In [0]:
-- Create a UDF for advanced PII masking
CREATE OR REPLACE FUNCTION secure_bank.mask_pii(input_string STRING, recipient STRING)
RETURNS STRING
LANGUAGE SQL
DETERMINISTIC
COMMENT 'Masks PII based on recipient access level'
RETURN 
  CASE 
    WHEN recipient IN ('global_auditor', 'compliance_team') THEN input_string
    WHEN recipient LIKE '%_analytics' THEN SHA2(input_string, 256)
    WHEN recipient LIKE 'marketing_%' THEN 
      CONCAT(SUBSTRING(input_string, 1, 2), REPEAT('*', LENGTH(input_string) - 2))
    ELSE '***REDACTED***'
  END;

In [0]:
-- Advanced customer view with UDF-based masking
CREATE OR REPLACE VIEW secure_bank.advanced_customer_view AS
SELECT 
  customer_id,
  secure_bank.mask_pii(first_name, current_recipient()) as first_name,
  secure_bank.mask_pii(last_name, current_recipient()) as last_name,
  secure_bank.mask_pii(email, current_recipient()) as email,
  secure_bank.mask_pii(phone, current_recipient()) as phone,
  secure_bank.mask_pii(ssn, current_recipient()) as ssn,
  -- Date masking - show year only for some recipients
  CASE 
    WHEN current_recipient() IN ('global_auditor', 'compliance_team') THEN date_of_birth
    WHEN current_recipient() LIKE '%_analytics' THEN DATE(CONCAT(YEAR(date_of_birth), '-01-01'))
    ELSE NULL
  END as date_of_birth,
  city,
  state,
  country,
  region,
  -- Financial data with noise injection for analytics
  CASE 
    WHEN current_recipient() IN ('global_auditor', 'compliance_team') THEN account_balance
    WHEN current_recipient() LIKE '%_analytics' THEN 
      account_balance + (RAND() * 1000 - 500)  -- Add small random noise
    ELSE NULL
  END as account_balance,
  credit_score,
  risk_category,
  account_type,
  customer_since,
  is_vip,
  data_classification
FROM secure_bank.customers
WHERE 
  -- Dynamic access control based on data classification and recipient
  CASE 
    WHEN data_classification = 'Public' THEN true
    WHEN data_classification = 'Internal' AND 
         current_recipient() NOT LIKE 'external_%' THEN true
    WHEN data_classification = 'Confidential' AND 
         current_recipient() IN ('global_auditor', 'compliance_team', 'risk_analytics_firm') THEN true
    WHEN data_classification = 'Restricted' AND 
         current_recipient() IN ('global_auditor', 'compliance_team') THEN true
    ELSE false
  END;

## Step 6: Create Recipients and Share Configuration

Set up different types of recipients to demonstrate the access controls.

In [0]:
-- Create different recipient types for testing
-- North America Partner
CREATE RECIPIENT IF NOT EXISTS na_partner
USING ID 'aws:us-east-1:test-na-partner-id'
COMMENT 'North America regional partner for customer analytics';

-- Europe Partner  
CREATE RECIPIENT IF NOT EXISTS eu_partner
USING ID 'aws:eu-west-1:test-eu-partner-id'
COMMENT 'Europe regional partner for customer analytics';

-- Marketing Agency
CREATE RECIPIENT IF NOT EXISTS marketing_agency_a
USING ID 'aws:us-west-2:test-marketing-agency-id'
COMMENT 'Marketing agency with limited demographic access';

-- Risk Analytics Firm
CREATE RECIPIENT IF NOT EXISTS risk_analytics_firm
USING ID 'aws:us-east-1:test-risk-analytics-id'
COMMENT 'Risk analytics firm with anonymized financial data access';

-- Global Auditor
CREATE RECIPIENT IF NOT EXISTS global_auditor
USING ID 'aws:us-east-1:test-global-auditor-id'
COMMENT 'Global auditor with full access and audit trails';

SHOW RECIPIENTS;

In [0]:
-- Create shares for different access patterns
CREATE SHARE IF NOT EXISTS regional_customer_data
COMMENT 'Regional customer data with geographic filtering';

CREATE SHARE IF NOT EXISTS marketing_safe_data
COMMENT 'Marketing-safe customer demographics without PII';

CREATE SHARE IF NOT EXISTS risk_analytics_data
COMMENT 'Anonymized financial data for risk modeling';

CREATE SHARE IF NOT EXISTS compliance_audit_data
COMMENT 'Full access data for compliance and audit purposes';

SHOW SHARES;

In [0]:
-- Add views to appropriate shares
-- Regional data share
ALTER SHARE regional_customer_data 
ADD VIEW secure_bank.regional_customers_view
COMMENT 'Regional customer view with dynamic filtering';

ALTER SHARE regional_customer_data 
ADD VIEW secure_bank.regional_transactions_view
COMMENT 'Regional transaction view with amount masking';

-- Marketing data share
ALTER SHARE marketing_safe_data 
ADD VIEW secure_bank.marketing_customers_view
COMMENT 'Marketing-safe customer demographics';

-- Risk analytics share
ALTER SHARE risk_analytics_data 
ADD VIEW secure_bank.risk_analytics_view
COMMENT 'Anonymized financial patterns for risk modeling';

-- Compliance audit share
ALTER SHARE compliance_audit_data 
ADD VIEW secure_bank.compliance_audit_view
COMMENT 'Full access with audit metadata';

ALTER SHARE compliance_audit_data 
ADD VIEW secure_bank.advanced_customer_view
COMMENT 'Advanced customer view with dynamic masking';

In [0]:
-- Grant access to different recipients
-- Regional partners get regional data
GRANT SELECT ON SHARE regional_customer_data TO RECIPIENT na_partner;
GRANT SELECT ON SHARE regional_customer_data TO RECIPIENT eu_partner;

-- Marketing agencies get marketing-safe data
GRANT SELECT ON SHARE marketing_safe_data TO RECIPIENT marketing_agency_a;

-- Risk analytics firm gets anonymized financial data
GRANT SELECT ON SHARE risk_analytics_data TO RECIPIENT risk_analytics_firm;

-- Global auditor gets everything
GRANT SELECT ON SHARE compliance_audit_data TO RECIPIENT global_auditor;
GRANT SELECT ON SHARE regional_customer_data TO RECIPIENT global_auditor;
GRANT SELECT ON SHARE marketing_safe_data TO RECIPIENT global_auditor;
GRANT SELECT ON SHARE risk_analytics_data TO RECIPIENT global_auditor;

## Step 7: Test Views with Simulated Recipients

Since we can't easily simulate recipient context in this environment, let's create test versions of our views.

In [0]:
-- Create test views that simulate different recipient access
-- Test: North America Partner View
CREATE OR REPLACE VIEW secure_bank.test_na_partner_view AS
SELECT 
  customer_id,
  first_name,
  last_name,
  email,
  city,
  state,
  country,
  region,
  account_balance,
  credit_score,
  account_type,
  customer_since,
  is_vip
FROM secure_bank.customers
WHERE region = 'North America';

SELECT 'NA Partner View' as view_type, COUNT(*) as record_count
FROM secure_bank.test_na_partner_view;

In [0]:
-- Test: Marketing Agency View
CREATE OR REPLACE VIEW secure_bank.test_marketing_view AS
SELECT 
  customer_id,
  SHA2(CONCAT(first_name, last_name), 256) as customer_hash,
  SUBSTRING(email, 1, 3) || '***@' || SUBSTRING_INDEX(email, '@', -1) as masked_email,
  city,
  state,
  country,
  region,
  CASE 
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 25 THEN '18-24'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 35 THEN '25-34'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 45 THEN '35-44'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 55 THEN '45-54'
    WHEN DATEDIFF(current_date(), date_of_birth) / 365 < 65 THEN '55-64'
    ELSE '65+'
  END as age_group,
  CASE 
    WHEN account_balance < 50000 THEN 'Basic'
    WHEN account_balance < 100000 THEN 'Standard'
    WHEN account_balance < 200000 THEN 'Premium'
    ELSE 'Private'
  END as account_tier,
  account_type,
  YEAR(customer_since) as customer_since_year,
  is_vip
FROM secure_bank.customers
WHERE data_classification IN ('Public', 'Internal');

SELECT * FROM secure_bank.test_marketing_view LIMIT 5;

In [0]:
-- Test: Risk Analytics View
CREATE OR REPLACE VIEW secure_bank.test_risk_analytics_view AS
SELECT 
  SHA2(customer_id, 256) as anonymous_customer_id,
  region,
  CASE 
    WHEN account_balance < 25000 THEN 'Low'
    WHEN account_balance < 100000 THEN 'Medium'
    WHEN account_balance < 250000 THEN 'High'
    ELSE 'Very High'
  END as balance_category,
  CASE 
    WHEN credit_score < 600 THEN 'Poor'
    WHEN credit_score < 650 THEN 'Fair'
    WHEN credit_score < 700 THEN 'Good'
    WHEN credit_score < 750 THEN 'Very Good'
    ELSE 'Excellent'
  END as credit_category,
  risk_category,
  account_type,
  FLOOR(DATEDIFF(current_date(), customer_since) / 365) as tenure_years,
  is_vip
FROM secure_bank.customers;

SELECT * FROM secure_bank.test_risk_analytics_view LIMIT 5;

## Step 8: Monitor and Audit Access Patterns

Create monitoring queries to track how different recipients are accessing data.

In [0]:
-- Create access log table for monitoring
CREATE TABLE IF NOT EXISTS secure_bank.access_audit_log (
  log_id STRING,
  recipient_name STRING,
  share_name STRING,
  view_name STRING,
  access_timestamp TIMESTAMP,
  row_count LONG,
  filter_applied STRING,
  data_classification_accessed STRING
)
USING DELTA
COMMENT 'Audit log for Delta Sharing access patterns';

In [0]:
-- Simulate access patterns for demonstration
INSERT INTO secure_bank.access_audit_log VALUES
  ('LOG001', 'na_partner', 'regional_customer_data', 'regional_customers_view', '2025-01-20 09:00:00', 3, 'region=North America', 'Internal,Confidential'),
  ('LOG002', 'marketing_agency_a', 'marketing_safe_data', 'marketing_customers_view', '2025-01-20 10:30:00', 6, 'data_classification IN (Public,Internal)', 'Public,Internal'),
  ('LOG003', 'risk_analytics_firm', 'risk_analytics_data', 'risk_analytics_view', '2025-01-20 11:15:00', 10, 'anonymized=true', 'All'),
  ('LOG004', 'global_auditor', 'compliance_audit_data', 'compliance_audit_view', '2025-01-20 14:20:00', 10, 'none', 'All'),
  ('LOG005', 'eu_partner', 'regional_customer_data', 'regional_customers_view', '2025-01-20 15:45:00', 3, 'region=Europe', 'Internal,Confidential');

-- Access pattern analysis
SELECT 
  recipient_name,
  COUNT(*) as access_count,
  SUM(row_count) as total_rows_accessed,
  MIN(access_timestamp) as first_access,
  MAX(access_timestamp) as last_access
FROM secure_bank.access_audit_log
GROUP BY recipient_name
ORDER BY total_rows_accessed DESC;

## Step 9: Performance Optimization for Dynamic Views

Optimize the underlying tables and views for better performance with dynamic filtering.

In [0]:
-- Optimize base tables for query performance
OPTIMIZE secure_bank.customers
ZORDER BY (region, data_classification, customer_id);

OPTIMIZE secure_bank.transactions
ZORDER BY (region, customer_id, transaction_date);

-- Analyze table statistics
ANALYZE TABLE secure_bank.customers COMPUTE STATISTICS;
ANALYZE TABLE secure_bank.transactions COMPUTE STATISTICS;

In [0]:
-- Create summary statistics for monitoring
SELECT 
  'customers' as table_name,
  COUNT(*) as total_records,
  COUNT(DISTINCT region) as regions,
  COUNT(DISTINCT data_classification) as classification_levels
FROM secure_bank.customers
UNION ALL
SELECT 
  'transactions' as table_name,
  COUNT(*) as total_records,
  COUNT(DISTINCT region) as regions,
  NULL as classification_levels
FROM secure_bank.transactions;

## Summary

✅ **What we accomplished:**

1. Created comprehensive source data with PII, financial, and regional information
2. Implemented dynamic views using `current_recipient()` for row-level security
3. Applied column-level masking and data transformation based on recipient type
4. Used partition filtering for regional and classification-based access control
5. Created custom UDFs for advanced PII masking strategies
6. Set up multiple recipient types with appropriate access levels
7. Implemented audit logging and access monitoring
8. Optimized performance for dynamic filtering scenarios

**Key Security Patterns Demonstrated:**
- 🔒 **Row-Level Security**: Regional and classification-based filtering
- 🎭 **Data Masking**: PII redaction, hashing, and bucketing
- 📊 **Anonymization**: Customer IDs, financial amounts, dates
- 🎯 **Conditional Access**: Different data views per recipient type
- 📝 **Audit Trails**: Access logging and monitoring
- ⚡ **Performance**: Optimized partitioning and indexing

**Access Control Matrix:**
| Recipient Type | PII Access | Financial Data | Regional Filter | Data Classification |
|---|---|---|---|---|
| Regional Partners | Limited | Yes | Region-specific | Internal+ |
| Marketing Agencies | Masked | No | All regions | Public/Internal |
| Risk Analytics | Anonymized | Bucketed | All regions | All levels |
| Global Auditors | Full | Full | All regions | All levels |

**Next Steps:**
Continue to the recipient notebook to see how these access controls appear to different recipient types.

&copy; 2025 Databricks, Inc. All rights reserved.